In [ ]:
import glob
import json
import os
import random
import shutil

import librosa
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import soundfile as sf 

import data_processing as dp
import plots

# listen to audios
import IPython.display as ipd

In [ ]:
%load_ext autoreload
%autoreload 2

Helper functions

In [ ]:
def load_shapley_values(row):
    question_id = row.Index
    data = f"../{row.output_folder}/{question_id}_info.npz"
    tokens = row.input_ids
    audio_tokens = np.where(tokens < 0)[-1]
    question_tokens = np.where(tokens >=0)[-1]

    all_shapley_values = data["shapley_values"].squeeze(0).squeeze(0)
    audio_shapley_values = all_shapley_values[audio_tokens]
    question_shapley_values = all_shapley_values[question_tokens]

def row_to_json(row, dataset_path, exp_info):
    if row.experiment.startswith("qwen"):
        sample_rate = 16000
    else:
        sample_rate = 24000

    x, fs = librosa.load(os.path.join(dataset_path, row.audio_path), sr=sample_rate)
    #all_shapley_values, audio_shapley_values, question_shapley_values = dp.load_shapley_values(sample)
    output = {}
    output["title"] = row.id
    all_shapley_values, audio_shapley_values, question_shapley_values = dp.load_shapley_values(row)
    
    output["all_shapley_values"] = all_shapley_values.tolist()
    output["audio_shapley_values"] = audio_shapley_values.tolist()
    output["question_shapley_values"] = question_shapley_values.tolist()
    output["question_tokens"] = row.input_tokens.tolist()
    output["answer_tokens"] = row.output_tokens.tolist()
    output["audio_signal"] = x.tolist()
    output["total_duration"] = len(x)/sample_rate
    
    output["num_shapley_samples"] = audio_shapley_values.shape[0]
    output["sample_rate"] = sample_rate
    output["gt_start"] = None
    output["gt_end"] = None

    output["model"] = exp_info[row.experiment]["model"]
    output["experiment"] = exp_info[row.experiment]["experiment"]

    return output

Load data

In [ ]:
dataset_path = "/media/gigibs/DD02EEEC68459F17/datasets"
# dataset_path = "/scratch/gv2167/datasets/"
#dataset_path = "/home/gigibs/Documents/datasets"

demo_path = "/home/gigibs/Documents/2025_investigating_mmshap_demo/data"

In [ ]:
qfs = pd.read_json("../data/output_data/qwen_fs.json")
qzs = pd.read_json("../data/output_data/qwen_zs.json")
mfs = pd.read_json("../data/output_data/mu_fs.json")
mzs = pd.read_json("../data/output_data/mu_zs.json")

In [ ]:
qtoy = pd.read_json("../data/output_data/qwen_toy.json")

In [ ]:
qtoy

In [ ]:
qfs = dp.parse_df(qfs, experiment_name="qwen_fs")
qzs = dp.parse_df(qzs, experiment_name="qwen_zs")
mfs = dp.parse_df(mfs, experiment_name="mu_fs")
mzs = dp.parse_df(mzs, experiment_name="mu_zs")

Setup experiment info and select questions to use as demo data

In [ ]:
exp_info = {
    "qwen_fs": {"model": "Qwen-Audio", "experiment": "MC-PI"},
    "qwen_zs": {"model": "Qwen-Audio", "experiment": "MC-NPI"},
    "mu_fs": {"model": "MU-LLaMA", "experiment": "MC-PI"},
    "mu_zs": {"model": "MU-LLaMA", "experiment": "MC-NPI"},
}

In [ ]:
all_events = qfs.index.values
# separate the sse_events that we want to display
sse_events = [427, 719, 396, 869]

# get 4 random other examples
random.seed(42)
random_events = random.choices(list(set(all_events) - set(sse_events)), k=4)

In [ ]:
demo_qids = sse_events + random_events

question_list.json should have the following structure:

```json
[
  {
    "question_type": "sse",
    "audio_path": "data/audio/719.wav",
    "qid": 719,
    "prompt": "What sound effect can be heard in this piece?"
  },
...
]
```

where `prompt` is the question without any options, just to hint the viewer on what they are going to see in the comparison page.

In [ ]:
# build question_list.json
question_list = []
for i in sse_events:
    question = {
        "question_type": "sse",
        "audio_path": f"data/audio/{i}.wav",
        "qid": str(i),
        "prompt": qfs.loc[i]["prompt"].split("Question: ")[-1].split("\n")[0]
    }

    question_list.append(question)

    # copy audio to audio path
    shutil.copy(os.path.join(dataset_path, qfs.loc[i]["audio_path"]), 
                os.path.join(demo_path, f"{i}.wav")
               )

for i in random_events:
    question = {
        "question_type": "random",
        "audio_path": f"data/audio/{i}.wav",
        "qid": str(i),
        "prompt": qfs.loc[i]["prompt"].split("Question: ")[-1].split("\n")[0]
    }

    question_list.append(question)
    shutil.copy(os.path.join(dataset_path, qfs.loc[i]["audio_path"]), 
                os.path.join(demo_path, f"{i}.wav")
               )


with open(os.path.join(demo_path, "question_list.json"), "w") as f:
    json.dump(question_list, f)

Demo {qid}.json format

```json
{
    "title": "example", // maybe we can skip this too
    "question_tokens": [list, of, tokens],
    "question_shapley_values": [list, of, values],
    "sample_rate": 10,
    "total_duration": 10,
    "num_audio_samples": 16000,
    "audio_signal": [list, of, samples],
    "audio_shapley_values": [list, of, values],
    "gt_start": 0.1 (or None),
    "gt_end": 0.2 (or None),
}
```

--- 

MAP JSON -> DF

title -> question.id?

question_tokens -> input_tokens

text_shapley_tokens -> **(LOAD)** text shapley 

sample_rate -> **(INFER)** 16k or 24k depending on the model

total_duration -> **(INFER)** from audio samples and sample rate

audio_signal -> **(LOAD)** audio 

audio_shapley_values -> **(LOAD)** audio shapley 

gt_start, gt_end -> **(LOAD)** from sse if exists, or None

In [ ]:
demo_qids

In [ ]:
for exp in [qfs, qzs, mfs, mzs]:
    for i in exp.loc[demo_qids].itertuples():
        output = row_to_json(i, dataset_path, exp_info)

        # ugly ugly ugly BUT it works
        if i.Index == 427:
            output["gt_start"] = 2.5
            output["gt_end"] = 4
        elif i.Index == 719:
            output["gt_start"] = 5.5
            output["gt_end"] = 7
        elif i.Index == 396:
            output["gt_start"] = 1
            output["gt_end"] = 3
        elif i.Index == 869:
            output["gt_start"] = 0
            output["gt_end"] = 4
            
        print(os.path.join(demo_path, f"{i.Index}_{i.experiment}.json"))
        with open(os.path.join(demo_path, f"{i.Index}_{i.experiment}.json"), "w") as f:
            json.dump(output, f)

In [ ]:
with open(f"/home/gigibs/Documents/2025_investigating_mmshap_demo/data/{i.Index}_{i.experiment}.json", "r") as f:
    tmp = json.load(f)

In [ ]:
np.array(tmp["audio_shapley_values"]).shape

In [ ]:
tmp